In [1]:
!pip install topojson geopandas folium pyproj mgwr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.8/331.8 kB 23.2 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import topojson as tp
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
import io
import base64
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pyproj
import geopandas as gpd
import mgwr
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

We load merged_geo_df.csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/City Masters/Thesis/datasets for thesis/merged_geo_df.csv'
df = pd.read_csv(file_path)
print(df.head())


Mounted at /content/drive
   id       code                                           geometry  \
0 NaN  E05000650  POLYGON ((370295 414672, 372394 411758, 370557...   
1 NaN  E05000651  POLYGON ((375017 414993, 375017 414809, 375709...   
2 NaN  E05000652  POLYGON ((373586 410473, 375160 410817, 375327...   
3 NaN  E05000653  POLYGON ((372394 411758, 370295 414672, 373276...   
4 NaN  E05000654  POLYGON ((370557 411414, 372394 411758, 373157...   

          areaName      cx      cy  detached  semiDetached  terraced  \
0    Astley Bridge  370732  412836  0.966641      1.564969  1.334971   
1         Bradshaw  374479  412684  1.370174      1.727901  0.074846   
2       Breightmet  374513  409534 -0.194186      2.489098  1.312189   
3    Bromley Cross  371904  414137  2.609219      0.911970  1.104304   
4  Crompton Bolton  371830  410799 -0.455505      0.602654  3.717106   

      flats  ...       oil      wood     solid  renewable  communal  \
0 -0.145641  ... -0.442458 -0.202465 -0.199

In [4]:
print(df)

      id       code                                           geometry  \
0    NaN  E05000650  POLYGON ((370295 414672, 372394 411758, 370557...   
1    NaN  E05000651  POLYGON ((375017 414993, 375017 414809, 375709...   
2    NaN  E05000652  POLYGON ((373586 410473, 375160 410817, 375327...   
3    NaN  E05000653  POLYGON ((372394 411758, 370295 414672, 373276...   
4    NaN  E05000654  POLYGON ((370557 411414, 372394 411758, 373157...   
...   ..        ...                                                ...   
7624 NaN  W05001796  POLYGON ((353864 205106, 352671 203936, 353148...   
7625 NaN  W05001797  POLYGON ((353220 194117, 353029 193819, 352170...   
7626 NaN  W05001798  POLYGON ((349857 212516, 350763 213387, 351956...   
7627 NaN  W05001799  POLYGON ((347019 188542, 347043 188542, 347591...   
7628 NaN  W05001800  POLYGON ((352767 215154, 355271 214374, 355294...   

                    areaName      cx      cy  detached  semiDetached  \
0              Astley Bridge  370732  4

We use the same function that converts British National Grid (BNG) coordinates to WGS84 latitude and longitude coordinates by using the pyproj library to define a transformation from the BNG system (EPSG:27700) to the global latitude-longitude system (EPSG:4326). The function bng_to_latlng_vectorized applies this transformation to easting and northing coordinates in a DataFrame, resulting in new columns for latitude and longitude. This is useful for mapping and spatial analysis in the ML and regression models.

In [5]:
from pyproj import Transformer

def bng_to_latlng_vectorized(easting, northing):
    # Create a transformer object for BNG (EPSG:27700) to WGS84 (EPSG:4326)
    transformer = Transformer.from_crs('epsg:27700', 'epsg:4326', always_xy=True)
    # Transform the coordinates
    lon, lat = transformer.transform(easting, northing)
    return lat, lon

# Convert coordinates for the entire DataFrame
df['latitude'], df['longitude'] = bng_to_latlng_vectorized(df['cx'], df['cy'])

The code cell performs correlation analysis on the merged_geo_df dataset. It starts by selecting a target variable ('vGood') and identifying columns to exclude from the analysis. The correlation matrix for the remaining features is calculated and then visualized with a heatmap. It then focuses on the upper triangle of this matrix, identifying features with high correlation (above 0.90) and marking them for removal to reduce multicollinearity. Once that's done the features get dropped, and a new correlation matrix is recalculated and visualized. This process helps in understanding the relationships between features and refines the dataset by removing highly correlated variables to avoid multicollinearity.

In [6]:
# Assuming your main dataframe is df
target_df = df[['vGood']]
columns_to_exclude = ['vGood', 'good', 'fair', 'bad', 'vBad', 'id', 'code', 'geometry', 'areaName', 'cx', 'cy', 'latitude', 'longitude']

# First, filter out the columns you want to exclude
features_df = df.drop(columns=columns_to_exclude)

# Calculate the correlation matrix
corr_matrix = features_df.corr().abs()

# Plot initial correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Initial Correlation Matrix")
plt.show()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

# Create a mask for highlighting features to drop
mask = upper.applymap(lambda x: x > 0.90)

# Plot the upper triangle of the correlation matrix with highlighted features to drop
plt.figure(figsize=(12, 8))
sns.heatmap(upper, cmap='coolwarm', annot=True, mask=mask)
plt.title("Features to Drop (Correlation > 0.90)")
plt.show()

# Drop features
features_df.drop(columns=to_drop, inplace=True)

# Recalculate the correlation matrix
new_corr_matrix = features_df.corr().abs()

# Plot final correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(new_corr_matrix, annot=True, cmap='coolwarm')
plt.title("Final Correlation Matrix After Dropping Features")
plt.show()


Output hidden; open in https://colab.research.google.com to view.

Now we employ the LassoCV method from Scikit-Learn to perform feature selection on the remaining variables. We start by defining X as the feature set and y as the target variable. It then initializes a LassoCV model with 5-fold cross-validation, a range of alpha values which is the regularization strength, and a high iteration count for fitting. After fitting is complete, we extract the coefficients associated with each feature. LassoCV has the ability to reduce the less important feature coefficients to zero. This allows us to select features with non-zero coefficients as the most relevant for predicting the target variable. We then finish by printing the remaining variables after the rest has been filtered.

In [7]:
from sklearn.linear_model import LassoCV

X = (features_df)

# Target variable
y = target_df.values.ravel()

# Increase the range of alphas and the number of iterations
lasso = LassoCV(cv=5, random_state=0, max_iter=10000, alphas=10**np.linspace(-4, -0.5, 100)).fit(X, y)
# Get the feature names
feature_names = features_df.columns

# Extract the coefficients and set a threshold to select features
lasso_coef = lasso.coef_
selected_features = feature_names[lasso_coef != 0]

print("Selected features:", selected_features)


Selected features: Index(['detached', 'semiDetached', 'flats', 'sharedHouse', 'conversion',
       'commercial', 'caravan', 'dep0', 'dep1', 'management', 'professional',
       'admin', 'popDensity', 'level3', 'L8_L9', 'L15', 'v0', 'v2', 'v3plus',
       'ABBangladeshi', 'ABChinese', 'ABIndian', 'ABPakistani', 'ABAsian',
       'BBAfrican', 'BBCaribbean', 'whiteAndAsian', 'whiteAndBlackAfrican',
       'multiple', 'wBritish', 'wIrish', 'wRoma', 'wOther', 'other_ethnic',
       'none_heating', 'bottledGas', 'elec', 'oil', 'wood', 'solid',
       'communal', 'other_heating', '2plusRenewable'],
      dtype='object')


Now we use the Random Forest Regressor to identify and rank the most important features in a dataset for predicting a target variable. Starting by selecting features identified as significant by the previous LASSO model. A Random Forest Regressor is then trained with these features and the feature importances are extracted and sorted in descending order. The code defines a variable top_n_features to specify the number of top features to be displayed (set to 10) and prints out these top features along with their importance scores.

In [8]:
from sklearn.ensemble import RandomForestRegressor

# Filter the features DataFrame to only include selected features from LASSO
X_selected = features_df[selected_features]

# Train a Random Forest Regressor on the selected features
forest = RandomForestRegressor(random_state=42)
forest.fit(X_selected, y)

# Get feature importances from the Random Forest model
importances = forest.feature_importances_

# Sort the feature importances
indices = np.argsort(importances)[::-1]

# Define the number of top features to print
top_n_features = 10 # Set this to the number of features you want to include

# Print the top feature importances
print(f"Top {top_n_features} features based on Random Forest after pruning:")
for f in range(top_n_features):
    # Check if the number of features is less than top_n_features
    if f < len(indices):
        print(f"{f + 1}. Feature '{X_selected.columns[indices[f]]}' (Importance: {importances[indices[f]]})")
    else:
        break


Top 10 features based on Random Forest after pruning:
1. Feature 'professional' (Importance: 0.23601043922388146)
2. Feature 'management' (Importance: 0.22146719602578438)
3. Feature 'v0' (Importance: 0.11783174590431054)
4. Feature 'dep1' (Importance: 0.10012572041877037)
5. Feature 'wBritish' (Importance: 0.058459410433690455)
6. Feature 'semiDetached' (Importance: 0.03430861939738077)
7. Feature 'level3' (Importance: 0.025058004574711093)
8. Feature 'whiteAndAsian' (Importance: 0.021364857438531986)
9. Feature 'L8_L9' (Importance: 0.021052358365104922)
10. Feature 'L15' (Importance: 0.014187575396499568)


Now for the GWR model, the process begins by preparing the data, including extracting relevant features and converting them to numpy arrays for analysis. The core of the script utilizes Optuna, a hyperparameter optimization framework, to find the best settings for the GWR model. It optimizes kernel type and model family (Gaussian, Poisson, or Binomial) by minimizing the Akaike Information Criterion (AIC), a measure balancing model fit and complexity. After running a specified number of trials, we identify the best kernel type and family. Then, the optimal parameters are used to construct the final GWR model. The model, which accounts for spatial variability in data by adjusting relationships across different geographic locations, is then fitted to the data, with results  providing localized insights which will be transposed on the UK map. The start and end times of the Optuna study is also recorded, for efficiency records.

In [9]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import spglm.family
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import optuna
import time


# Remove non-numeric and duplicate columns
selected_features = ['professional', 'management', 'v0', 'dep1','wBritish', 'semiDetached', 'level3' ,'whiteAndAsian'
                     ,'L8_L9', 'L15']
geo_columns = ['latitude', 'longitude']
target_column = 'vGood'

# Create a new DataFrame with the selected features, geographic columns, and target
model_df = df[selected_features + geo_columns + [target_column]]

# Convert to numpy arrays
X = np.array(model_df[selected_features])
y = np.array(model_df[target_column]).reshape((-1, 1))
coords = np.array(model_df[geo_columns])

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to be optimized
    kernel_type = trial.suggest_categorical('kernel', ['bisquare', 'gaussian', 'exponential'])
    family_type = trial.suggest_categorical('family', ['Gaussian', 'Poisson', 'Binomial'])

    # Select model family
    if family_type == 'Gaussian':
        model_family = spglm.family.Gaussian()
    elif family_type == 'Poisson':
        model_family = spglm.family.Poisson()
    elif family_type == 'Binomial':
        model_family = spglm.family.Binomial()

    # Bandwidth selection
    bw = Sel_BW(coords, y, X, fixed=False, kernel=kernel_type)
    gwr_bw = bw.search()
    print(f"Optuna trial bandwidth: {gwr_bw}")

    # Create and fit GWR model
    model = GWR(coords, y, X, bw=gwr_bw, family=model_family, fixed=False, kernel=kernel_type)
    results = model.fit()

    # Return the metric to optimize, e.g., AIC
    return results.aic

# Start the timer
start_time = time.time()

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Adjust the number of trials as necessary

# Stop the timer
end_time = time.time()

# Calculate the duration
duration = end_time - start_time
print(f"Total duration for Optuna optimization: {duration} seconds")

# Output the best hyperparameters
best_kernel = study.best_params['kernel']
best_family = study.best_params['family']
print('Best kernel:', best_kernel)
print('Best family:', best_family)

# Create the final GWR model using the best parameters
if best_family == 'Gaussian':
    final_family = spglm.family.Gaussian()
elif best_family == 'Poisson':
    final_family = spglm.family.Poisson()
elif best_family == 'Binomial':
    final_family = spglm.family.Binomial()

bw = Sel_BW(coords, y, X, fixed=False, kernel=best_kernel)
gwr_bw = bw.search()
print(f"Final selected bandwidth: {gwr_bw}")
gwr_model = GWR(coords, y, X, bw=gwr_bw, family=final_family, fixed=False, kernel=best_kernel)
gwr_results = gwr_model.fit()




[I 2023-12-09 05:18:43,136] A new study created in memory with name: no-name-6d49900b-aaf6-4bd7-9868-0f5295276970


Optuna trial bandwidth: 118.0


[I 2023-12-09 05:33:57,015] Trial 0 finished with value: 2964.41514107883 and parameters: {'kernel': 'bisquare', 'family': 'Poisson'}. Best is trial 0 with value: 2964.41514107883.


Optuna trial bandwidth: 63.0


[I 2023-12-09 05:49:51,630] Trial 1 finished with value: 11024.05418741779 and parameters: {'kernel': 'exponential', 'family': 'Binomial'}. Best is trial 0 with value: 2964.41514107883.


...

[I 2023-12-09 07:17:19,331] Trial 7 finished with value: 1051.905491576354 and parameters: {'kernel': 'gaussian', 'family': 'Poisson'}. Best is trial 3 with value: -34295.99479375255.


Optuna trial bandwidth: 118.0


[I 2023-12-09 07:29:36,905] Trial 8 finished with value: -36806.50211064709 and parameters: {'kernel': 'bisquare', 'family': 'Gaussian'}. Best is trial 8 with value: -36806.50211064709.


Optuna trial bandwidth: 118.0


[I 2023-12-09 07:41:50,874] Trial 9 finished with value: -36806.50211064709 and parameters: {'kernel': 'bisquare', 'family': 'Gaussian'}. Best is trial 8 with value: -36806.50211064709.


Now we'll extract the local parameters of each feature used in the GWR model, and we will transpose this on the map to analyze how the features vary spatially.

In [ ]:
# Extract local parameter estimates
local_params = gwr_results.params

In [ ]:
# Create a DataFrame for coordinates
df_coords = pd.DataFrame(coords, columns=['latitude', 'longitude'])

# Set up the matplotlib figure
fig, axes = plt.subplots(4, 3, figsize=(15, 20))  # 4 rows, 3 columns
fig.subplots_adjust(hspace=0.4, wspace=0.4)  # Adjust horizontal and vertical spaces

# Iterate over features and plot
for i, feature in enumerate(selected_features):
    row = i // 3  # Determine the row index
    col = i % 3   # Determine the column index
    ax = axes[row, col]

    # Add the coefficients for the current feature
    df_coords['coefficient'] = local_params[:, i + 1]  # +1 to skip intercept

    # Plot using seaborn on the specified subplot axis
    sns.scatterplot(data=df_coords, x='longitude', y='latitude', hue='coefficient', palette='BrBG', ax=ax)
    ax.set_title(f"Coefficient for '{feature}'")
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

# If the number of features is not a multiple of 3, hide the empty subplots
for j in range(i + 1, 12):
    fig.delaxes(axes[j // 3, j % 3])

plt.show()


Now the results of the GWR model is extracted below, focusing on various scalar and array metrics. A dictionary is created to store scalar results like AIC Score and Residual Variance. For Gaussian models, it also includes R-squared and Adjusted R-squared metrics, plus a new DataFrame of Local R-Squared Values, which represent the model's explanatory power in each local wards. Standardized Residuals and the Residual Sum of Squares (RSS) are compiled into their respective DataFrames, providing insights into the model's prediction errors.

A new df is created for parameter estimates, showing the impact of each feature, including the intercept, across different locations, reflecting GWR's capability to model spatially-varying relationships. RMSE is calculated and added to the scalar results for a comprehensive overview. The updated scalar results are then displayed in a table format to provide a summary of the model's performance.

In [ ]:
# Create a dictionary for scalar results
scalar_results_dict = {
    "Metric": ["AIC Score", "Residual Variance"],
    "Value": [gwr_results.aic, gwr_results.sigma2]
}

# Check if the best model family is Gaussian before adding R-squared and Adjusted R-squared
if best_family == 'Gaussian':
    scalar_results_dict["Metric"].extend(["R-squared", "Adjusted R-squared"])
    scalar_results_dict["Value"].extend([gwr_results.R2, gwr_results.adj_R2])

    # Also, handle Local R-Squared Values only for Gaussian
    local_r2_df = pd.DataFrame(gwr_results.localR2, columns=["Local R-Squared Values"])
    print("\nLocal R-Squared Values:")
    print(local_r2_df)

# Create DataFrame for scalar results
scalar_results_df = pd.DataFrame(scalar_results_dict)

# Print the scalar results table
print("GWR Model Scalar Results:")
print(scalar_results_df)

# Handle array metrics
std_residuals_df = pd.DataFrame(gwr_results.std_res, columns=["Standardized Residuals"])
rss_df = pd.DataFrame(gwr_results.RSS, columns=["Residual Sum of Squares"])

# Print array metrics tables
print("\nStandardized Residuals:")
print(std_residuals_df)
print("\nResidual Sum of Squares (RSS):")
print(rss_df)

# Create the parameter estimates DataFrame
parameter_estimates_df = pd.DataFrame(gwr_results.params)

# Set column names for the parameter estimates DataFrame
feature_names = ['Intercept'] + selected_features
parameter_estimates_df.columns = feature_names

# Print the parameter estimates table
print("\nParameter Estimates:")
print(parameter_estimates_df)

# Calculate RMSE (Root Mean Squared Error)
# Access residuals based on model family
if best_family in ['Gaussian', 'Poisson']:
    residuals = gwr_results.resid_response
else: # For Binomial or other families
    residuals = gwr_results.resid_pearson

rmse = np.sqrt(np.mean(residuals**2))

# Add RMSE to the scalar results dictionary
scalar_results_dict["Metric"].append("RMSE")
scalar_results_dict["Value"].append(rmse)

# Update and Print the updated scalar results table
scalar_results_df = pd.DataFrame(scalar_results_dict)
print("Updated GWR Model Scalar Results:")
print(scalar_results_df)


Now we transform 'geometry' column of a DataFrame into geometric objects using the shapely library, which is essential for spatial analysis. The wkt.loads function is applied to each row in the 'geometry' column to convert Well-Known Text (WKT) format strings into shapely geometric objects. This transformation enables the DataFrame to be converted into a GeoDataFrame, which can then be used for advanced spatial analyses, visualizations, and operations that require geometric computations.

In [ ]:
from shapely import wkt
print(df['geometry'].head())
df['geometry'] = df['geometry'].apply(wkt.loads) #uncomment this line if error occurs

# Now create the GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')


Below we add results from the GWR model. We check if the GWR model uses a Gaussian family. The parameter estimates from the GWR model, including the y-intercept, are added as new columns in the GeoDataFrame. Additionally, it incorporates key model metrics such as residual variance, standardized residuals, and the AIC score. For Gaussian models, it also adds local R-squared values, Residual Sum of Squares (RSS), and Adjusted R-squared, providing a comprehensive spatial overview of the model's performance at different geographic locations.



In [ ]:
from shapely import wkt
import geopandas as gpd

# Determine model family type
is_gaussian = isinstance(gwr_model.family, spglm.family.Gaussian)
# Extract the parameter estimates from the GWR results
parameter_estimates = gwr_results.params

# Adding the y-intercept
gdf['Intercept'] = parameter_estimates[:, 0]

# Add the GWR results to the GeoDataFrame
gdf['residual_variance'] = gwr_results.sigma2
gdf['standardized_residuals'] = gwr_results.std_res
gdf['aic_score'] = gwr_results.aic

# Conditional additions for Gaussian models
if is_gaussian:
    gdf['local_r2'] = gwr_results.localR2
    gdf['residual_sum_of_squares'] = gwr_results.RSS  # Add Residual Sum of Squares
    gdf['adjusted_r2'] = gwr_results.adj_R2  # Add Adjusted R-squared

Now to visualize the raw risiduals, we convert the Peach color scheme from Plotly to hexadecimal format, which creates a matplotlib colormap with these hexadecimal colors. The raw residuals are calculated by subtracting the model's predicted values from the actual response values, and these residuals are added to the GeoDataFrame. We use matplotlib to plot the spatial distribution of these raw residuals on a map using the Peach colormap.

In [ ]:

import matplotlib.colors as mcolors
import plotly.express as px

# Get the Peach color scheme from Plotly
peach_palette = px.colors.diverging.BrBG

# Convert 'rgb()' format to hexadecimal
def rgb_to_hex(rgb_str):
    rgb = rgb_str.strip('rgb()').split(',')
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

peach_hex_palette = [rgb_to_hex(color) for color in peach_palette]

# Create a colormap using the converted hexadecimal colors
peach_cmap = mcolors.LinearSegmentedColormap.from_list("Peach", peach_hex_palette)

# Extract the raw residuals from the GWR results
# Ensure that raw_residuals is a one-dimensional array
raw_residuals = (gwr_results.resid_response.flatten() - gwr_results.predy.flatten())

# Add the raw residuals to the GeoDataFrame
gdf['raw_residuals'] = raw_residuals

# Plotting Raw Residuals using the Peach colormap
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='raw_residuals', cmap=peach_cmap, legend=True, ax=ax)
ax.set_title('GWR Spatial Distribution of Raw Residuals')
plt.show()


We create more graphs here by converting the RdYlGn (Red-Yellow-Green) color scheme from Plotly into hexadecimal format to create a matplotlib colormap. Using this colormap, we visualize various metrics like residual variance, standardized residuals, AIC score, and for Gaussian models, local R-squared values, Residual Sum of Squares (RSS), and Adjusted R-squared.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import plotly.express as px

# Get the BrBG color scheme from Plotly
brbg_palette = px.colors.diverging.BrBG

# Convert 'rgb()' format to hexadecimal
def rgb_to_hex(rgb_str):
    rgb = rgb_str.strip('rgb()').split(',')
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

brbg_hex_palette = [rgb_to_hex(color) for color in brbg_palette]

# Create a colormap using the converted hexadecimal colors
brbg_cmap = mcolors.LinearSegmentedColormap.from_list("BrBG", brbg_hex_palette)

# Plotting Residual Variance
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='residual_variance', cmap=brbg_cmap, legend=True, ax=ax)
ax.set_title('GWR Spatial Distribution of Residual Variance')
plt.show()

# Plotting Standardized Residuals
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='standardized_residuals', cmap='BrBG', legend=True, ax=ax,
         vmin=-4, vmax=4)  # Set the range from -4 to 4
ax.set_title('GWR Spatial Distribution of Standardized Residuals')
plt.show()
# Plotting AIC Score
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='aic_score', cmap=brbg_cmap, legend=True, ax=ax)
ax.set_title('GWR Spatial Distribution of AIC Score')
plt.show()

# Conditional plotting for Gaussian models
if is_gaussian:
    # Plotting Local R-Squared
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf.plot(column='local_r2', cmap=brbg_cmap, legend=True, ax=ax)
    ax.set_title('GWR Spatial Distribution of Local R-Squared')
    plt.show()

    # Plotting Residual Sum of Squares
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf.plot(column='residual_sum_of_squares', cmap=brbg_cmap, legend=True, ax=ax)
    ax.set_title('GWR Spatial Distribution of Residual Sum of Squares')
    plt.show()

    # Plotting Adjusted R-Squared
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf.plot(column='adjusted_r2', cmap=brbg_cmap, legend=True, ax=ax)
    ax.set_title('GWR Spatial Distribution of Adjusted R-Squared')
    plt.show()


Finally, we create a Dash app. We set up a dropdown menu and a map plot in the app layout, using the previously mentioned GWR metrics and the local coefficients of the features used in the GWR model. The app allows users to select a metric from the dropdown, and the map will update to display the spatial distribution of the selected metric using Plotly Express, a plotting library that works seamlessly with Dash. This interactivity makes it easy to explore and understand the spatial aspects of the GWR model's results, providing a user-friendly interface for analyzing complex geospatial data.

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

# Initialize the Dash app
app = dash.Dash(__name__)

# GWR metrics with updated column names and parameter estimates
gwr_metrics = {
    'Residual Variance': 'residual_variance',
    'Standardized Residuals': 'standardized_residuals',
    'AIC Score': 'aic_score'
}

# Add Gaussian-specific metrics if applicable
if is_gaussian:
    gwr_metrics['Local R-Squared'] = 'local_r2'
    gwr_metrics['Residual Sum Squares'] = 'residual_sum_of_squares'
    gwr_metrics['Adjusted R-Squared'] = 'adjusted_r2'

# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='metric-dropdown',
        options=[{'label': k, 'value': v} for k, v in gwr_metrics.items()],
        value='local_r2'  # Default metric
    ),
    dcc.Graph(id='gwr-map', style={'height': '85vh', 'width': '100%'})
])

# Callback to update the map
@app.callback(
    Output('gwr-map', 'figure'),
    Input('metric-dropdown', 'value')
)
def update_map(selected_metric):
    fig = px.scatter_mapbox(
        gdf,
        lat="latitude",
        lon="longitude",
        hover_name='areaName',  # Replace with the correct column name
        color=selected_metric,
        color_continuous_scale=px.colors.diverging.BrBG,
        size_max=20,
        zoom=5,
        mapbox_style="carto-darkmatter"
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, autosize=True)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Assuming df_coeffs is your DataFrame with latitude, longitude, and coefficients
# Example structure: 'latitude', 'longitude', 'professional_coeff', 'management_coeff', ...

# List of features for which you have coefficients
selected_features = ['professional', 'management', 'v0', 'dep1', 'wBritish', 'semiDetached', 'level3', 'whiteAndAsian', 'L8_L9', 'L15']

# Create a DataFrame for coordinates and coefficients
df_coeffs = pd.DataFrame(coords, columns=['latitude', 'longitude'])
for i, feature in enumerate(selected_features, start=1):  # Start from 1 to skip intercept
    df_coeffs[f'{feature}_coeff'] = local_params[:, i]

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='feature-dropdown',
        options=[{'label': f'Coefficient of {feature}', 'value': f'{feature}_coeff'} for feature in selected_features],
        value=f'{selected_features[0]}_coeff'  # Default to the first feature's coefficient
    ),
    dcc.Graph(id='gwr-coeff-map', style={'height': '85vh', 'width': '100%'})
])

# Callback to update the map
@app.callback(
    Output('gwr-coeff-map', 'figure'),
    Input('feature-dropdown', 'value')
)
def update_map(selected_coeff):
    fig = px.scatter_mapbox(
        df_coeffs,
        lat="latitude",
        lon="longitude",
        color=selected_coeff,
        color_continuous_scale=px.colors.diverging.BrBG,
        size_max=15,
        zoom=5,
        mapbox_style="carto-darkmatter"
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, autosize=True)
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


Our notebook for GWR ends here.